In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import pyplot
from random import randint
import math

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.utils.multiclass import unique_labels
from scipy.spatial import distance
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import export_graphviz
from sklearn import tree
from sklearn import metrics
from sklearn import tree
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn import linear_model
from sklearn import neighbors
from sklearn.utils import resample
from sklearn.metrics import cohen_kappa_score

%matplotlib inline




In [2]:
import io
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
train_data = pd.read_csv('/content/drive/My Drive/Dataset/fashion-mnist_train.csv')
test_data = pd.read_csv('/content/drive/My Drive/Dataset/fashion-mnist_test.csv')

In [0]:
def create_classifier(classifier_type, tree_min_samples_split = 20):

    if classifier_type == "svm":
        c = svm.SVC(probability=True)

    elif classifier_type == "logreg":
        c = linear_model.LogisticRegression(multi_class='ovr', solver='liblinear', max_iter=1000)

    elif classifier_type == "knn":
        c = neighbors.KNeighborsClassifier()

    elif classifier_type == "tree":
        c = tree.DecisionTreeClassifier(min_samples_split = tree_min_samples_split)

    elif classifier_type == "randomforest":
        c = ensemble.RandomForestClassifier()
        
    else:
        c = linear_model.LogisticRegression(multi_class='ovr', solver='liblinear', max_iter=1000)
    
    return c

In [0]:
#create a sample of train_data for fast training
data_sampling_rate = 0.1
train_data = train_data.sample(frac=data_sampling_rate)
target = "label"
X = [i for i in train_data.columns if i not in target]
X = train_data[X]
y = train_data[target]

In [0]:
# Create a new classifier which is based on the sckit-learn BaseEstimator and ClassifierMixin classes
class StackedEnsembleClassifier(BaseEstimator, ClassifierMixin):
    
    """An ensemble classifier that uses heterogeneous models at the base layer and a aggregatnio model at the aggregation layer. A k-fold cross validation is used to gnerate training data for the stack layer model.

    Parameters
    ----------
    base_estimators: list 
        A list of the classifiers in the ase layer of the ensemble. Supported types are
        - "svm" Support Vector Machine implemented by sklearn.svm.SVC
        - "logreg" Logistic Regression implemented by sklearn.linear_models.LogisticRegression
        - "knn" k Nearest Neighbour implemented by sklearn.neighbors.KNeighborsClassifier
        - "tree" Decision Tree implemented by sklearn.tree.DecisionTreeClassifier
        - "randomforest" RandomForest implemented by sklearn.tree.RandomForestClassifier    
    classifier_duplicates: int, optional (default = 1)
        How many instances of each classifier type listed in base_estimators is included in the ensemble
    stack_layer_classifier: string, optional (default = "logreg')
        The classifier type used at the stack layer. The same classifier types as are supported at the base layer are supported        
    training_folds: int, optional (default = 4)
        How many folds will be used to generate the training set for the stacked layer
        
    Attributes
    ----------
    classes_ : array of shape = [n_classes] 
        The classes labels (single output problem).


    Notes
    -----
    The default values for most base learners are used.

    See also
    --------
    
    ----------
    .. [1]  van der Laan, M., Polley, E. & Hubbard, A. (2007). 
            Super Learner. Statistical Applications in Genetics 
            and Molecular Biology, 6(1) 
            doi:10.2202/1544-6115.1309
    Examples
    --------
    >>> from sklearn.datasets import load_iris
    >>> from sklearn.model_selection import cross_val_score
    >>> clf = StackedEnsembleClassifier()
    >>> iris = load_iris()
    >>> cross_val_score(clf, iris.data, iris.target, cv=10)

    """
    # Constructor for the classifier object
    def __init__(self, base_estimator_types = ["svm", "logreg", "tree"], base_estimator_duplicates = 8, stack_layer_classifier_type = "logreg"):
        """Setup a SuperLearner classifier .
        Parameters
        ----------
        base_estimator_types: The types of classifiers to include at the base layer
        base_estimator_duplicates: The number of duplicates of each type of classiifer to include
        stack_layer_classifier_type: The type of classifier to include at the stack layer 
        
        Returns
        -------
        Nothing
        """     

        # Initialise class variabels
        self.base_estimator_types = base_estimator_types
        self.base_estimator_type_list = list()
        self.base_estimator_duplicates = base_estimator_duplicates
        self.stack_layer_classifier_type = stack_layer_classifier_type

    # The fit function to train a classifier
    def fit(self, X, y):
        """Build a SuperLearner classifier from the training set (X, y).
        Parameters
        ----------
        X : array-like, shape = [n_samples, n_features]
            The training input samples. 
        y : array-like, shape = [n_samples] 
            The target values (class labels) as integers or strings.
        Returns
        -------
        self : object
        """    
        
        # Check that X and y have correct shape
        X, y = check_X_y(X, y)
        # Store the classes seen during fit
        self.classes_ = unique_labels(y)
        
        ########################
        # LEVEL 0
        ########################
        
        # Set up the base classifeirs in the ensemble
        self.classifiers_ = list()
        
        for i in range(0, self.base_estimator_duplicates):
            for t in self.base_estimator_types:

                self.base_estimator_type_list.append(t)      
                c = create_classifier(t, tree_min_samples_split=math.ceil(len(X)*0.05))
                self.classifiers_.append(c)
        
        # Store the number of classifers in the ensemble
        self.n_estimators_ = len(self.classifiers_)

        # Use all training data to train base classifiers
        X_train = X
        y_train = y
        
        # Set up empty arrays to hold stack layer training data
        self.X_stack_train = None #(dtype = float)
        self.y_stack_train = y_train
          
        # Train each base calssifier and generate the stack layer training dataset
        for classifier in self.classifiers_:

            # Extract a bootstrap sample
            X_train_samp, y_train_samp = resample(X_train, y_train, replace=True)    
            
            # Train a base classifier
            classifier.fit(X_train_samp, y_train_samp)
            
            # Make predictions for all instances in the training set
            y_pred = classifier.predict_proba(X_train)

            # Append the predictions ot the stack layer traing set (a bit of hacking here!)
            try:
                self.X_stack_train = np.c_[self.X_stack_train, y_pred]
            except ValueError:
                self.X_stack_train = y_pred
      
        ########################
        # LEVEL 1
        ########################
        
        # Create the stack layer classifier
        self.stack_layer_classifier_ = create_classifier(self.stack_layer_classifier_type, tree_min_samples_split=math.ceil(len(X)*0.05))

        # Train the stack layer using the newly created dataset
        self.stack_layer_classifier_.fit(self.X_stack_train, self.y_stack_train)
            
        # Return the classifier
        return self

    # The predict function to make a set of predictions for a set of query instances
    def predict(self, X):
        """Predict class labels of the input samples X.
        Parameters
        ----------
        X : array-like matrix of shape = [n_samples, n_features]
            The input samples. 
        Returns
        -------
        p : array of shape = [n_samples, ].
            The predicted class labels of the input samples. 
        """
        
        # Check is fit had been called by confirming that the teamplates_ dictiponary has been set up
        check_is_fitted(self, ['stack_layer_classifier_'])

        # Check that the input features match the type and shape of the training features
        X = check_array(X)
   
        X_stack_queries = None
              
        # Make a prediction with each base classifier and assemble the stack layer query
        for classifier in self.classifiers_:
            
            y_pred = classifier.predict_proba(X)
            
            try:
                X_stack_queries = np.c_[X_stack_queries, y_pred]
            except ValueError:
                X_stack_queries = y_pred
        
        # Return the prediction made by the stack layer classifier
        return self.stack_layer_classifier_.predict(X_stack_queries)
    
    # The predict function to make a set of predictions for a set of query instances
    def predict_proba(self, X):
        """Predict class probabilities of the input samples X.
        Parameters
        ----------
        X : array-like matrix of shape = [n_samples, n_features]
            The input samples. 
        Returns
        -------
        p : array of shape = [n_samples, n_labels].
            The predicted class label probabilities of the input samples. 
        """
        # Check is fit had been called by confirming that the teamplates_ dictiponary has been set up
        check_is_fitted(self, ['stack_layer_classifier_'])

        # Check that the input features match the type and shape of the training features
        X = check_array(X)
        
        X_stack_queries = None
        
        # Make a prediction with each base classifier
        for classifier in self.classifiers_:
            
            y_pred = classifier.predict_proba(X)
                
            try:
                X_stack_queries = np.c_[X_stack_queries, y_pred]
            except ValueError:
                X_stack_queries = y_pred

        # Return the prediction made by the stack layer classifier        
        return self.stack_layer_classifier_.predict_proba(X_stack_queries)

In [0]:
class StackedEnsembleHoldOut(BaseEstimator, ClassifierMixin):
  # Constructor for the classifier object
  def __init__(self, base_estimator_types = ["svm", "logreg", "tree"], base_estimator_duplicates = 2, stack_layer_classifier_type = "logreg"):
    # Initialise class variabels
    self.base_estimator_types = base_estimator_types
    self.base_estimator_type_list = list()
    self.base_estimator_duplicates = base_estimator_duplicates
    self.stack_layer_classifier_type = stack_layer_classifier_type

  # The fit function to train a classifier
  def fit(self, X, y):
    # Check that X and y have correct shape
    X, y = check_X_y(X, y)
        
    # Store the classes seen during fit
    self.classes_ = unique_labels(y)
        
    # Set up the base classifeirs in the ensemble
    self.classifiers_ = list()
    
    for i in range(0, self.base_estimator_duplicates):
      
      for t in self.base_estimator_types:
        
        self.base_estimator_type_list.append(t)      
        c = create_classifier(t, tree_min_samples_split=math.ceil(len(X)*0.05))
        self.classifiers_.append(c)
        
    # Store the number of classifers in the ensemble
    self.n_estimators_ = len(self.classifiers_)
        
    # Set up empty arrays to hold stack layer training data
    self.X_stack_train = None #(dtype = float)
    self.y_stack_train = None
    
    #create a hold out set
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0,\
                                                              train_size = 0.8)
        
    # Append the true value of hold out set to y_stack_train
    try:
      self.y_stack_train = np.c_[self.y_stack_train, y_test]
    except ValueError:
      self.y_stack_train = y_test #* might need to change
    
    for classifier in self.classifiers_:
      # Train a base classifier
      classifier.fit(X_train, y_train)
      
      # Make predictions for all instances in the hold out set
      y_pred = classifier.predict_proba(X_test)

      # Append the predictions ot the stack layer traing set (a bit of hacking here!)
      try:
        self.X_stack_train = np.c_[self.X_stack_train, y_pred]
      except ValueError:
        self.X_stack_train = y_pred #* might need to change
    
    # Create the stack layer classifier
    self.stack_layer_classifier_ = create_classifier(self.stack_layer_classifier_type, tree_min_samples_split=math.ceil(len(X)*0.05))

    # Train the stack layer using the newly created dataset
    self.stack_layer_classifier_.fit(self.X_stack_train, self.y_stack_train)
                
    return self
    
  # The predict function to make a set of predictions for a set of query instances
  def predict(self, X):
    """Predict class labels of the input samples X.
    Parameters
    ----------
    X : array-like matrix of shape = [n_samples, n_features]
    
    The input samples. 
    Returns
    -------
    p : array of shape = [n_samples, ].
            The predicted class labels of the input samples. 
        """
        
    # Check is fit had been called by confirming that the teamplates_ dictiponary has been set up
    check_is_fitted(self, ['stack_layer_classifier_'])

    # Check that the input features match the type and shape of the training features
    X = check_array(X)
   
    X_stack_queries = None
              
    # Make a prediction with each base classifier and assemble the stack layer query
    for classifier in self.classifiers_:
      y_pred = classifier.predict_proba(X)
      
      try:
        X_stack_queries = np.c_[X_stack_queries, y_pred]
      except ValueError:
        X_stack_queries = y_pred
        
    # Return the prediction made by the stack layer classifier
    return self.stack_layer_classifier_.predict(X_stack_queries)
      
  # The predict function to make a set of predictions for a set of query instances
  def predict_proba(self, X):
    """Predict class probabilities of the input samples X.
    Parameters
    ----------
    X : array-like matrix of shape = [n_samples, n_features]
            The input samples. 
    Returns
    -------
    p : array of shape = [n_samples, n_labels].
         The predicted class label probabilities of the input samples. 
        """
    # Check is fit had been called by confirming that the teamplates_ dictiponary has been set up
    check_is_fitted(self, ['stack_layer_classifier_'])

    # Check that the input features match the type and shape of the training features
    X = check_array(X)
        
    X_stack_queries = None
        
    # Make a prediction with each base classifier
    for classifier in self.classifiers_:
      y_pred = classifier.predict_proba(X)
      
      try:
        X_stack_queries = np.c_[X_stack_queries, y_pred]
      except ValueError:
        X_stack_queries = y_pred

    # Return the prediction made by the stack layer classifier        
    return self.stack_layer_classifier_.predict_proba(X_stack_queries)
    


In [47]:
clf = StackedEnsembleHoldOut()
clf.fit(X, y)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


StackedEnsembleHoldOut(base_estimator_duplicates=2,
            base_estimator_types=['svm', 'logreg', 'tree'],
            stack_layer_classifier_type='logreg')

In [48]:
x_hold = pd.DataFrame(clf.X_stack_train)
x_hold

,0,1,2,3,4,5
0,3,4,4,3,4,4
1,3,3,3,3,3,3
2,3,2,4,3,2,4
3,3,8,5,3,8,5
4,3,0,0,3,0,0
5,3,4,2,3,4,2
6,3,1,1,3,1,1
7,3,8,8,3,8,8
8,3,9,9,3,9,9
9,3,4,4,3,4,4
